The purpose of this notebook is to create a function that takes the imported player list:
- clean to only needed information
- seperate by pither and batter
- merge each on player name to get projections from model
- merge back pitcher and batter data 
- output df for now

Next steps:
- build line up based on salary

In [46]:
# imports
import pandas as pd
import numpy as np

In [47]:
# will need fanduel import
fd = pd.read_csv('../../../Downloads/FanDuel-MLB-2021 ET-05 ET-11 ET-58668-players-list.csv')

In [48]:
# print first 5 rows of fd
fd.head()

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Probable Pitcher,Batting Order,Roster Position
0,58668-52859,P,Jacob,Jacob deGrom,deGrom,57.333333,6.0,12500,BAL@NYM,NYM,BAL,O,Side,NaN,NaN,0.0,P
1,58668-16956,P,Gerrit,Gerrit Cole,Cole,50.857143,7.0,12300,NYY@TB,NYY,TB,NaN,NaN,NaN,NaN,NaN,P
2,58668-5481,P,Max,Max Scherzer,Scherzer,45.428571,7.0,12000,PHI@WSH,WSH,PHI,NaN,NaN,NaN,NaN,0.0,P
3,58668-52183,P,Tyler,Tyler Glasnow,Glasnow,47.250000,8.0,11500,NYY@TB,TB,NYY,NaN,NaN,NaN,NaN,0.0,P
4,58668-60633,P,Carlos,Carlos Rodon,Rodon,52.200000,5.0,11200,MIN@CWS,CWS,MIN,NaN,NaN,NaN,NaN,0.0,P


only columns needed:
- id - will need this later for template
- Position
- nickname - renamed to Name
- salary
- game
- team
- opponent
- injury indicator
- probable pitcher

In [49]:
fd.columns

Index(['Id', 'Position', 'First Name', 'Nickname', 'Last Name', 'FPPG',
       'Played', 'Salary', 'Game', 'Team', 'Opponent', 'Injury Indicator',
       'Injury Details', 'Tier', 'Probable Pitcher', 'Batting Order',
       'Roster Position'],
      dtype='object')

In [50]:
# clean up fd to match column list above
fd.drop(columns=['First Name', 'Last Name', 'FPPG', 'Played',
                 'Injury Details', 'Tier', 'Batting Order', 'Roster Position'], inplace=True)

Next step is fill nulls in probable pitcher and injury indicator

In [51]:
# filling nulls for probable pitcher
fd['Probable Pitcher'].fillna('No', inplace=True)

In [52]:
# fill nulls for injury indicator
fd['Injury Indicator'].fillna('Healthy', inplace=True)

In [53]:
# review new cleaned df
fd.head()

,Id,Position,Nickname,Salary,Game,Team,Opponent,Injury Indicator,Probable Pitcher
0,58668-52859,P,Jacob deGrom,12500,BAL@NYM,NYM,BAL,O,No
1,58668-16956,P,Gerrit Cole,12300,NYY@TB,NYY,TB,Healthy,No
2,58668-5481,P,Max Scherzer,12000,PHI@WSH,WSH,PHI,Healthy,No
3,58668-52183,P,Tyler Glasnow,11500,NYY@TB,TB,NYY,Healthy,No
4,58668-60633,P,Carlos Rodon,11200,MIN@CWS,CWS,MIN,Healthy,No


In [54]:
fd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 939 entries, 0 to 938
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Id                939 non-null    object
 1   Position          939 non-null    object
 2   Nickname          939 non-null    object
 3   Salary            939 non-null    int64 
 4   Game              939 non-null    object
 5   Team              939 non-null    object
 6   Opponent          939 non-null    object
 7   Injury Indicator  939 non-null    object
 8   Probable Pitcher  939 non-null    object
dtypes: int64(1), object(8)
memory usage: 66.1+ KB


Next step rename Nickname to name

In [55]:
# renaming nickname column
fd.rename(columns={'Nickname': 'Name'}, inplace=True)

Next step filter to only healthy players

In [56]:
# fitler to only healthy players
fd = fd.loc[fd['Injury Indicator'] == 'Healthy']

In [57]:
# review dataframe
fd.head()

,Id,Position,Name,Salary,Game,Team,Opponent,Injury Indicator,Probable Pitcher
1,58668-16956,P,Gerrit Cole,12300,NYY@TB,NYY,TB,Healthy,No
2,58668-5481,P,Max Scherzer,12000,PHI@WSH,WSH,PHI,Healthy,No
3,58668-52183,P,Tyler Glasnow,11500,NYY@TB,TB,NYY,Healthy,No
4,58668-60633,P,Carlos Rodon,11200,MIN@CWS,CWS,MIN,Healthy,No
6,58668-16959,P,Trevor Bauer,11000,SEA@LAD,LAD,SEA,Healthy,No


In [58]:
fd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 755 entries, 1 to 938
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Id                755 non-null    object
 1   Position          755 non-null    object
 2   Name              755 non-null    object
 3   Salary            755 non-null    int64 
 4   Game              755 non-null    object
 5   Team              755 non-null    object
 6   Opponent          755 non-null    object
 7   Injury Indicator  755 non-null    object
 8   Probable Pitcher  755 non-null    object
dtypes: int64(1), object(8)
memory usage: 59.0+ KB


next step is to split into pithcers and batters

In [59]:
# split using .loc by position and make new dataframe for pitchers
pitchers = fd.loc[fd['Position']=='P']

In [60]:
# split using .loc by position and make new dataframe for batters
batters = fd.loc[fd['Position']!='P']

Now we have cleaned dataframes for each group. Now lets focus on pitchers to get projections added to dataframe.

First step for pitchers is to get the probable pitchers first, we only care about the ones that will start.

In [61]:
# save pitchers df to only starting pitchers
pitchers = pitchers.loc[pitchers['Probable Pitcher']=='Yes']

In [62]:
# review changes
pitchers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22 entries, 7 to 179
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Id                22 non-null     object
 1   Position          22 non-null     object
 2   Name              22 non-null     object
 3   Salary            22 non-null     int64 
 4   Game              22 non-null     object
 5   Team              22 non-null     object
 6   Opponent          22 non-null     object
 7   Injury Indicator  22 non-null     object
 8   Probable Pitcher  22 non-null     object
dtypes: int64(1), object(8)
memory usage: 1.7+ KB


Next step is to combine projections with 2021 stats.
- import testing data with projections from model
- merge the two data frames

In [63]:
# read in pitcher projections
pitcher_proj = pd.read_csv('../Projections/pitcher_projections_2021.csv')

Next step merge.

In [64]:
# merge attempt
pitcher_projections = pitchers.merge(pitcher_proj, how='left', on='Name')

Next step is to drop nulls from this list.  Will need to handle this in previous processes for final product.

In [65]:
pitcher_projections.dropna(inplace=True)

Next step is to clean up and only take what is needed from pitchers so that it can be merged with batters.
Columns needed:
- ID
- Position
- Name
- Salary
- Team
- Opponent
- Proj_FPPG

In [66]:
# overwrite df with only the columns needed
pitcher_projections = pitcher_projections[['Id', 'Position', 'Name', 'Salary', 'Team_x', 'Opponent', 'Projected_FPPG']]

In [67]:
# rename team column
pitcher_projections.rename(columns={'Team_x' : 'Team'}, inplace=True)

Now we have a cleaned and organized df of our pitchers that are starting. Next is the batters.

In [68]:
pitcher_projections

,Id,Position,Name,Salary,Team,Opponent,Projected_FPPG
0,58668-97300,P,John Means,11000,BAL,NYM,29.788501
1,58668-79168,P,Walker Buehler,10800,LAD,SEA,15.475341
2,58668-53067,P,Chris Bassitt,9200,OAK,BOS,30.490788
4,58668-80433,P,Dinelson Lamet,9000,SD,COL,12.701435
5,58668-83190,P,Freddy Peralta,8600,MIL,STL,35.355267
6,58668-52159,P,Lance McCullers Jr.,8500,HOU,LAA,33.684272
7,58668-39041,P,Marcus Stroman,8400,NYM,BAL,22.284910
8,58668-53029,P,Robbie Ray,8300,TOR,ATL,25.044635
9,58668-79091,P,Dylan Cease,8200,CWS,MIN,29.891094
10,58668-63136,P,Matthew Boyd,8100,DET,KC,23.732519


---

First step is to merge on projections.

In [69]:
# read in projections file
batter_21 = pd.read_csv('../Projections/batter_projections_2021.csv')

In [70]:
# merge projections with batter df, creating new df
batters_projections = batters.merge(batter_21, how='left', on='Name')

Will have to drop nulls for now, but need to figure out why there is no data for those batters.

In [71]:
# dropping batters with no projections
batters_projections.dropna(inplace=True)

In [72]:
batters_projections

,Id,Position,Name,Salary,Game,Team_x,Opponent,Injury Indicator,Probable Pitcher,Team_y,...,CS,TB,AVG,OBP,SLG,OPS,ISO,PTS,FPPG,Projected_FPPG
0,58668-79393,SS,Fernando Tatis Jr.,4700,SD@COL,SD,COL,Healthy,No,SD,...,1.0,45.0,0.240,0.326,0.600,0.926,0.360,301.0,15.050000,9.721812
2,58668-12933,OF,Mike Trout,4400,LAA@HOU,LAA,HOU,Healthy,No,LAA,...,0.0,62.0,0.412,0.515,0.775,1.290,0.363,357.0,15.521739,12.139403
3,58668-82575,OF,Yordan Alvarez,4300,LAA@HOU,HOU,LAA,Healthy,No,HOU,...,0.0,44.0,0.312,0.360,0.550,0.910,0.238,250.3,11.919048,8.593627
4,58668-13540,OF,J.D. Martinez,4200,OAK@BOS,BOS,OAK,Healthy,No,BOS,...,0.0,72.0,0.347,0.431,0.713,1.144,0.366,419.2,15.525926,11.281578
5,58668-39151,OF,Mookie Betts,4100,SEA@LAD,LAD,SEA,Healthy,No,LAD,...,2.0,39.0,0.250,0.367,0.424,0.791,0.174,258.2,11.736364,7.707638
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,58668-102341,OF,DJ Peters,2000,SEA@LAD,LAD,SEA,Healthy,No,LAD,...,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.0,0.000000,-0.159347
348,58668-81869,C,Chad Wallach,2000,MIA@ARI,MIA,ARI,Healthy,No,MIA,...,0.0,13.0,0.282,0.310,0.333,0.643,0.051,65.4,4.671429,5.139429
349,58668-12237,C,Jonathan Lucroy,2000,PHI@WSH,WSH,PHI,Healthy,No,WSH,...,0.0,6.0,0.357,0.357,0.429,0.786,0.072,25.0,5.000000,6.212545
350,58668-102358,SS,Jose Devers,2000,MIA@ARI,MIA,ARI,Healthy,No,MIA,...,0.0,1.0,0.111,0.100,0.111,0.211,0.000,9.7,1.940000,1.435586


In [73]:
# drop unneeded columns for merge with pitcher, overwrite current df
batters_projections = batters_projections[['Id', 'Position', 'Name', 'Salary', 'Team_x', 'Opponent','Projected_FPPG']]

In [74]:
# rename team column
batters_projections.rename(columns={'Team_x' : 'Team'}, inplace=True)

C:\Users\phill\anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


Now there are two clean dataframes with the same columns, now we will determine optimal line up.

Steps to get optimal line up:
1. set salary cap
2. sort by fppg

    a. choose pitcher - based on projected fppg; in final product need to incoporate stats with strong coefficient values
    b. choose batters based on remaining salary and highest fppg; in final product need to incoporate stats with strong coefficient values
    
3. create list to append choices to
4. save to one df for output
5. take df and save to templet for export

In [36]:
# set cap for fanduel
salary_cap = 35_000

In [77]:
# sort pitcher by fppg projections
pitcher_projections.sort_values(by='Projected_FPPG', ascending=False, inplace=True, ignore_index=True)

In [39]:
# create a player list to 
lineup = []
lineup.append(pitcher_projections.values[0])

In [40]:
# need to update remaining salary
salary_cap -= pitcher_projections['Salary'][0]

In [42]:
# with updated salary fill remaining roster based on position and highest fppg
# create position list for remaining roster spots
position_list = ['C', '1B', '2B', '3B', 'SS', 'OF', 'OF', 'OF']
# sort batters by FPPG
batters_projections.sort_values(by='Projected_FPPG', ascending=False, inplace=True, ignore_index=True)

<ipython-input-42-fa0ceb13b7ac>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batters_projections.sort_values(by='Projected_FPPG', ascending=False, inplace=True, ignore_index=True)


In [43]:
# redo 
# create count based on remaining positions
sal_count = 8

# create average salary variable for remaining players
avg_sal = salary_cap/sal_count

# create for loop for each position in list to take highest fppg
for pos in position_list:
    # setting counter to increase if player is already in list
    # this is inside the for loop beacuse it needs to be per position
    counter = 0
    # if salary greater than average move to next player
    for salary in batters_projections.loc[batters_projections['Position'] == pos]['Salary']:
        # test if salary is greater than average if it is increase counter
        if batters_projections.loc[batters_projections['Position'] == pos]['Salary'].values[counter] > avg_sal:
            counter += 1
        else:
            # if less than average add player to list
            lineup.append(batters_projections.loc[batters_projections['Position'] == pos].values[counter])
            # drop player so no duplicates are added
            batters_projections.drop(batters_projections.loc[batters_projections['Position'] == pos].index.values[counter], inplace=True)
            # decrease sal_count
            sal_count -= 1
            # decrease salary cap
            salary_cap -= batters_projections.loc[batters_projections['Position'] == pos]['Salary'].values[counter]
            # create new average salary
            avg_sal = salary_cap/sal_count
            break


C:\Users\phill\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-43-595d937b3060>:28: RuntimeWarning: divide by zero encountered in longlong_scalars
  avg_sal = salary_cap/sal_count


In [44]:
df = pd.DataFrame(lineup, columns=['Id', 'Position', 'Name', 'Salary', 'Team', 'Opponent','Proj_FPPG'])
df

,Id,Position,Name,Salary,Team,Opponent,Proj_FPPG
0,58668-83190,P,Freddy Peralta,8600,MIL,STL,35.355267
1,58668-79174,C,Jacob Nottingham,2300,MIL,STL,26.105398
2,58668-52175,1B,Matt Olson,3200,OAK,BOS,9.676589
3,58668-68587,2B,Ozzie Albies,3200,ATL,TOR,7.963934
4,58668-5104,3B,Pablo Sandoval,2000,ATL,TOR,9.393400
5,58668-38614,SS,Carlos Correa,3000,HOU,LAA,7.521995
6,58668-82533,OF,Daulton Varsho,2000,ARI,MIA,10.760021
7,58668-38973,OF,Tyrone Taylor,2400,MIL,STL,8.650956
8,58668-60643,OF,Aaron Judge,3600,NYY,TB,9.199638


In [45]:
df['Salary'].sum()

30300